In [7]:
%pylab inline

import re
import numpy as np
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [8]:
# Function to parse regulatory adjusted cohort graduation rate which returns a data frame
def parsecohortdata(x, y):
    # Get the regulatory adjusted cohort graduation rate for all district
    acrg = pd.read_csv(x, dtype={"LEAID": str})

    #Add the Yar column
    acrg['YEAR'] = y

    #Rename existing coulmns
    acrg.columns = ['STATE', 'FIPST', 'LEAID', 'LEANM', 'ALL_COHORT', 'ALL_RATE', 
                    'MAM_COHORT', 'MAM_RATE', 'MAS_COHORT', 'MAS_RATE', 'MBL_COHORT', 
                    'MBL_RATE', 'MHI_COHORT', 'MHI_RATE', 'MTR_COHORT', 'MTR_RATE',
                    'MWH_COHORT', 'MWH_RATE', 'CWD_COHORT', 'CWD_RATE', 'ECD_COHORT', 
                    'ECD_RATE', 'LEP_COHORT', 'LEP_RATE', 'DATE_CUR', 'YEAR']

    acrg = acrg.applymap(cleandata)
    acrg = acrg.fillna(0)
    
    #Convert all column data to numeric data
    acrg['ALL_RATE'] = pd.to_numeric(acrg['ALL_RATE'])
    acrg['MAM_RATE'] = pd.to_numeric(acrg['MAM_RATE'])
    acrg['MAS_RATE'] = pd.to_numeric(acrg['MAS_RATE'])
    acrg['MBL_RATE'] = pd.to_numeric(acrg['MBL_RATE'])
    acrg['MHI_RATE'] = pd.to_numeric(acrg['MHI_RATE'])
    acrg['MTR_RATE'] = pd.to_numeric(acrg['MTR_RATE'])
    acrg['MWH_RATE'] = pd.to_numeric(acrg['MWH_RATE'])
    acrg['CWD_RATE'] = pd.to_numeric(acrg['CWD_RATE'])
    acrg['ECD_RATE'] = pd.to_numeric(acrg['ECD_RATE'])
    acrg['LEP_RATE'] = pd.to_numeric(acrg['LEP_RATE'])
    
    #Calculate the % of students from different race and round the data to the nearest int
    acrg['MAM_RATIO'] = round(acrg['MAM_COHORT']/acrg['ALL_COHORT']*100, 0)
    acrg['MAS_RATIO'] = round(acrg['MAS_COHORT']/acrg['ALL_COHORT']*100, 0)
    acrg['MBL_RATIO'] = round(acrg['MBL_COHORT']/acrg['ALL_COHORT']*100, 0)
    acrg['MHI_RATIO'] = round(acrg['MHI_COHORT']/acrg['ALL_COHORT']*100, 0)
    acrg['MTR_RATIO'] = round(acrg['MTR_COHORT']/acrg['ALL_COHORT']*100, 0)
    acrg['MWH_RATIO'] = round(acrg['MWH_COHORT']/acrg['ALL_COHORT']*100, 0)
    acrg['CWD_RATIO'] = round(acrg['CWD_COHORT']/acrg['ALL_COHORT']*100, 0)
    acrg['ECD_RATIO'] = round(acrg['ECD_COHORT']/acrg['ALL_COHORT']*100, 0)
    acrg['LEP_RATIO'] = round(acrg['LEP_COHORT']/acrg['ALL_COHORT']*100, 0)
    
    #return the dataframe
    return acrg[['YEAR', 'STATE', 'LEAID', 'ALL_COHORT', 'ALL_RATE', 'MAM_RATE', 'MAS_RATE', 'MBL_RATE', 'MHI_RATE', 'MTR_RATE',
                 'MWH_RATE', 'CWD_RATE', 'ECD_RATE', 'LEP_RATE', 'MAM_RATIO', 'MAS_RATIO', 'MBL_RATIO', 
                'MHI_RATIO', 'MTR_RATIO', 'MWH_RATIO', 'CWD_RATIO', 'ECD_RATIO', 'LEP_RATIO']]

In [9]:
def cleandata(x):
    if (isinstance(x, str)):
        if re.search("[P][S]", x) and len(x) == 2:
            #z = pd.to_numeric(re.sub('[PS]', '', x))
            y = np.random.randint(0,100)

            return y
        if re.search("[L][T][0-9][0-9]", x) and len(x) == 4:
            z = pd.to_numeric(re.sub('[LT]', '', x))
            y = np.random.randint(0,z)
            return y
        
        if re.search("[L][E][0-9][0-9]", x) and len(x) == 4:
            z = pd.to_numeric(re.sub('[LE]', '', x))
            y = np.random.randint(0,z)
            return y
        
        if re.search("[L][E][0-9]", x) and len(x) == 3:
            z = pd.to_numeric(re.sub('[LE]', '', x))
            y = np.random.randint(0,z)
            return y
        
        if re.search("[G][E][0-9][0-9]", x) and len(x) == 4:
            z = pd.to_numeric(re.sub('[GE]', '', x))
            y = np.random.randint(z,100)

            return y
        
        if re.search("[G][T][0-9][0-9]", x) and len(x) == 4:
            z = pd.to_numeric(re.sub('[GT]', '', x))
            y = np.random.randint(z,100)

            return y
        
        if re.search("[0-9][0-9]-[0-9][0-9]", x) and len(x) == 5:
            z = x.split('-')
            a = pd.to_numeric(z[0])
            b = pd.to_numeric(z[1])
            y = np.random.randint(a,b)
            
            return y
        
        if re.search("[0-9]-[0-9]", x) and len(x) == 3:
            z = x.split('-')
            a = pd.to_numeric(z[0])
            b = pd.to_numeric(z[1])
            y = np.random.randint(a,b)
            
            return y
    return x

In [10]:
#Get the Public Elementary–Secondary Education Finance Data for all district
def parsefinancedata(x):
    sefd = pd.read_csv(x, encoding='latin-1', dtype={"NCESID": str})
    
    #Rename Column
    sefd.rename(columns={'NCESID':'LEAID'}, inplace=True)
    
    #Select a subset of columns
    sefd = sefd[['LEAID', 'ENROLL', 'TOTALREV', 'TFEDREV', 'TSTREV', 'TLOCREV']]
    
    #Calculate the Ratio of each source
    sefd['FED_RATIO'] = round(sefd['TFEDREV']/sefd['TOTALREV']*100, 0)
    sefd['ST_RATIO'] = round(sefd['TSTREV']/sefd['TOTALREV']*100, 0)
    sefd['LOC_RATIO'] = round(sefd['TLOCREV']/sefd['TOTALREV']*100, 0)
    
    #Return the dataframe
    return sefd[['LEAID', 'FED_RATIO', 'ST_RATIO', 'LOC_RATIO']]

In [11]:
#Get the Small Area Income and Poverty Estimates for all district
def parsepovertydata(x):
    smipe = pd.read_csv(x, encoding='latin-1', dtype={"State FIPS Code": str, "District ID": str})

    #Add a new column as LEAID = 'State FIPS Code' + 'District ID'
    smipe['LEAID'] = smipe['State FIPS Code'] + smipe['District ID']

    #Create a new Dataframe with selected column
    smipe = smipe[['LEAID', 'Total Population', 'Child Population', 'Child Population In Poverty']]
    
    #Replace invalid data in the dataframe
    smipe['Total Population'] = smipe['Total Population'].replace(',', '', regex=True)
    smipe['Child Population'] = smipe['Child Population'].replace(',', '', regex=True)
    smipe['Child Population In Poverty'] = smipe['Child Population In Poverty'].replace(',', '', regex=True)

    #Convert all column data in the dataframe to numeric data
    smipe['Total Population'] = pd.to_numeric(smipe['Total Population'])
    smipe['Child Population'] = pd.to_numeric(smipe['Child Population'])
    smipe['Child Population In Poverty'] = pd.to_numeric(smipe['Child Population In Poverty'])
    
    #Round the numeric data to nearest int
    smipe['CHILD_RATIO'] = round(smipe['Child Population']/smipe['Total Population']*100, 0)
    smipe['POVERTY_RATIO'] = round(smipe['Child Population In Poverty']/smipe['Child Population']*100, 0)
    
    #Return dataframe
    return smipe[['LEAID', 'CHILD_RATIO', 'POVERTY_RATIO']]

In [12]:
#Get the state Assessment data in Reading/Language Arts and Mathematics
def parseassessmentdata(x, year, subject):
    dt = pd.read_csv(x, dtype = str, na_values=['n/a'])
    z = str(year % 2000)
    y = str(year % 2000) + str((year % 2000)+1)
    
    #Add the Year column
    dt['YEAR'] = year
    
    if (subject == 'MATH'):
        s = 'MTH'
    
    if (subject == 'ENGLISH'):
        s = 'RLA'
        
    dt.columns = [x.upper() for x in dt.columns]
    
    #Rename existing coulmns in the regulatory adjusted cohort graduation rate dataframe
    dt.rename(columns={'STNAM':'STATE', 'LEANM' + z:'LEANM', 
                       'ALL_' + s + 'HSNUMVALID_' + y:'ALL_' + s + '_NUM', 'ALL_' + s + 'HSPCTPROF_' + y:'ALL_' + s + '_PCT',
                       'MAM_' + s + 'HSNUMVALID_' + y:'MAM_' + s + '_NUM', 'MAM_' + s + 'HSPCTPROF_' + y:'MAM_' + s + '_PCT',
                       'MAS_' + s + 'HSNUMVALID_' + y:'MAS_' + s + '_NUM', 'MAS_' + s + 'HSPCTPROF_' + y:'MAS_' + s + '_PCT',
                       'MBL_' + s + 'HSNUMVALID_' + y:'MBL_' + s + '_NUM', 'MBL_' + s + 'HSPCTPROF_' + y:'MBL_' + s + '_PCT',
                       'MHI_' + s + 'HSNUMVALID_' + y:'MHI_' + s + '_NUM', 'MHI_' + s + 'HSPCTPROF_' + y:'MHI_' + s + '_PCT',
                       'MTR_' + s + 'HSNUMVALID_' + y:'MTR_' + s + '_NUM', 'MTR_' + s + 'HSPCTPROF_' + y:'MTR_' + s + '_PCT',
                       'MWH_' + s + 'HSNUMVALID_' + y:'MWH_' + s + '_NUM', 'MWH_' + s + 'HSPCTPROF_' + y:'MWH_' + s + '_PCT',
                       'F_' + s + 'HSNUMVALID_' + y :'F_' + s + '_NUM',   'F_' + s + 'HSPCTPROF_' + y :'F_' + s + '_PCT',
                       'M_' + s + 'HSNUMVALID_' + y :'M_' + s + '_NUM',   'M_' + s + 'HSPCTPROF_' + y :'M_' + s + '_PCT',
                       'CWD_' + s + 'HSNUMVALID_' + y:'CWD_' + s + '_NUM', 'CWD_' + s + 'HSPCTPROF_' + y:'CWD_' + s + '_PCT',
                       'ECD_' + s + 'HSNUMVALID_' + y:'ECD_' + s + '_NUM', 'ECD_' + s + 'HSPCTPROF_' + y:'ECD_' + s + '_PCT',
                       'LEP_' + s + 'HSNUMVALID_' + y:'LEP_' + s + '_NUM', 'LEP_' + s + 'HSPCTPROF_' + y:'LEP_' + s + '_PCT',
                      }, inplace=True)

    dt  = dt[['STATE', 'FIPST', 'LEAID', 'LEANM', 'ALL_' + s + '_NUM', 'ALL_' + s + '_PCT', 'MAM_' + s + '_NUM', 'MAM_' + s + '_PCT',
              'MAS_' + s + '_NUM', 'MAS_' + s + '_PCT', 'MBL_' + s + '_NUM', 'MBL_' + s + '_PCT', 'MHI_' + s + '_NUM', 'MHI_' + s + '_PCT', 
              'MTR_' + s + '_NUM', 'MTR_' + s + '_PCT', 'MWH_' + s + '_NUM', 'MWH_' + s + '_PCT', 'F_' + s + '_NUM', 'F_' + s + '_PCT',
              'M_' + s + '_NUM', 'M_' + s + '_PCT', 'CWD_' + s + '_NUM', 'CWD_' + s + '_PCT', 'ECD_' + s + '_NUM', 'ECD_' + s + '_PCT',
              'LEP_' + s + '_NUM', 'LEP_' + s + '_PCT']]

    #Replace invalid data in the dataframe
    for col in dt.columns.values:
        #print(col)
        dt[col] = dt[col].replace('[GE]', '', regex=True)
        dt[col] = dt[col].replace('[PS]', 0, regex=True)
        dt[col] = dt[col].replace('[LT]', '', regex=True)   
        dt[col] = dt[col].replace('[0-9]*-', '', regex=True)
        dt[col] = dt[col].fillna(0)
        dt[col].replace(' ', '')
        dt[col].replace('[NaN]', '')

    #Convert all column data to numeric data
    numeric = ['ALL_' + s + '_PCT','MAM_' + s + '_PCT','MAS_' + s + '_PCT','MBL_' + s + '_PCT','MHI_' + s + '_PCT','MWH_' + s + '_PCT','F_' + s + '_PCT',
              'M_' + s + '_PCT', 'CWD_' + s + '_PCT', 'ECD_' + s + '_PCT','LEP_' + s + '_PCT']

    dt[numeric] = dt[numeric].apply(pd.to_numeric, errors='coerce')

    dt.fillna(0)
    return dt[['LEAID', 'ALL_' + s + '_PCT','MAM_' + s + '_PCT','MAS_' + s + '_PCT',
               'MBL_' + s + '_PCT','MHI_' + s + '_PCT','MWH_' + s + '_PCT','F_' + s + '_PCT',
               'M_' + s + '_PCT', 'CWD_' + s + '_PCT', 'ECD_' + s + '_PCT','LEP_' + s + '_PCT']]

In [13]:
#Get the National Center for Education Statistics for all district
def parseelsidata(x, y):
    elsi = pd.read_csv(x, dtype=str)

    #Add the Year column
    elsi['YEAR'] = y

    #Rename existing coulmns
    elsi.columns = ['STATE', 'LEAID', 'LEANM', 'Total Schools', 'Total Charter Schools', 'Total Public Schools', 
                    'Total Students', 'Free and Reduced Lunch Students', 'Grade 12 Students - male',
                    'Grade 12 Students - female', 'Pupil/Teacher Ratio', 'Full-Time Equivalent', 
                    'Secondary Teachers', 'Total Staff', 'Secondary Guidance Counselors', 'YEAR']

    for col in elsi.columns.values:
            elsi[col] = elsi[col].astype('str') 
            elsi[col] = elsi[col].replace('"', '')
            elsi[col] = elsi[col].replace('†', 0)
            elsi[col] = elsi[col].replace('–', 0)
            elsi[col] = elsi[col].replace('‡', 0)
            elsi[col] = elsi[col].replace('=', '')
            elsi[col] = elsi[col].fillna(0)
            elsi[col].replace('[NaN]', '')

    elsi['Total Schools'] = pd.to_numeric(elsi['Total Schools'])
    elsi['Total Charter Schools'] = pd.to_numeric(elsi['Total Charter Schools'])
    elsi['Total Public Schools'] = pd.to_numeric(elsi['Total Public Schools'])
    elsi['Total Students'] = pd.to_numeric(elsi['Total Students'])
    elsi['Free and Reduced Lunch Students'] = pd.to_numeric(elsi['Free and Reduced Lunch Students'])
    elsi['Grade 12 Students - male'] = pd.to_numeric(elsi['Grade 12 Students - male'])
    elsi['Grade 12 Students - female'] = pd.to_numeric(elsi['Grade 12 Students - female'])
    elsi['Pupil/Teacher Ratio'] = pd.to_numeric(elsi['Pupil/Teacher Ratio'])
    elsi['Full-Time Equivalent'] = pd.to_numeric(elsi['Full-Time Equivalent'])
    elsi['Secondary Teachers'] = pd.to_numeric(elsi['Secondary Teachers'])
    elsi['Total Staff'] = pd.to_numeric(elsi['Total Staff'])
    elsi['Secondary Guidance Counselors'] = pd.to_numeric(elsi['Secondary Guidance Counselors'])

    #Round the data to the nearest int
    elsi['CHARTER_SCHOOL_RATIO'] = round(elsi['Total Charter Schools']/elsi['Total Schools']*100, 0)
    elsi['PUBLIC_SCHOOL_RATIO'] = round(elsi['Total Public Schools']/elsi['Total Schools']*100, 0)
    elsi['FREE_AND_REDUCED_LUNCH_RATIO'] = round(elsi['Free and Reduced Lunch Students']/elsi['Total Students']*100, 0)
    elsi['SECONDARY_TEACHERS_RATIO'] = round(elsi['Secondary Teachers']/elsi['Full-Time Equivalent']*100, 0)
    elsi['SECONDARY_COUNSELORS_RATIO'] = round(elsi['Secondary Guidance Counselors']/elsi['Total Staff']*100, 0)

    elsi[elsi==np.inf] = np.nan
    elsi = elsi.fillna(0)

    return elsi[['LEAID', 'CHARTER_SCHOOL_RATIO', 'PUBLIC_SCHOOL_RATIO', 'FREE_AND_REDUCED_LUNCH_RATIO', 
                 'SECONDARY_TEACHERS_RATIO', 'SECONDARY_COUNSELORS_RATIO']]

In [14]:
A1 = pd.read_csv("data/EDFacts/regulatory-adjusted-cohort-graduation-rate/2011/acgr-lea-sy2011-12.csv", dtype=str)
A2 = pd.read_csv("data/NCES/2011/ELSI_csv_export_2011.csv", dtype=str)
A3 = pd.read_csv("data/EDFacts/Assessment Proficiency/2011/math-achievement-lea-sy2011-12.csv", dtype=str)
A4 = pd.read_csv("data/EDFacts/Assessment Proficiency/2011/rla-achievement-lea-sy2011-12.csv", dtype=str)

B1 = pd.read_csv("data/EDFacts/regulatory-adjusted-cohort-graduation-rate/2014/acgr-lea-sy2014-15.csv", dtype=str)
B2 = pd.read_csv("data/NCES/2014/ELSI_csv_export_2014.csv", dtype=str)
B3 = pd.read_csv("data/EDFacts/Assessment Proficiency/2014/math-achievement-lea-sy2014-15.csv", dtype=str)
B4 = pd.read_csv("data/EDFacts/Assessment Proficiency/2014/rla-achievement-lea-sy2014-15.csv", dtype=str)

In [15]:
print(A1.columns)
print(B1.columns)

Index(['STNAM', 'FIPST', 'LEAID', 'LEANM', 'ALL_COHORT_1112', 'ALL_RATE_1112',
       'MAM_COHORT_1112', 'MAM_RATE_1112', 'MAS_COHORT_1112', 'MAS_RATE_1112',
       'MBL_COHORT_1112', 'MBL_RATE_1112', 'MHI_COHORT_1112', 'MHI_RATE_1112',
       'MTR_COHORT_1112', 'MTR_RATE_1112', 'MWH_COHORT_1112', 'MWH_RATE_1112',
       'CWD_COHORT_1112', 'CWD_RATE_1112', 'ECD_COHORT_1112', 'ECD_RATE_1112',
       'LEP_COHORT_1112', 'LEP_RATE_1112', 'DATE_CUR'],
      dtype='object')
Index(['STNAM', 'FIPST', 'LEAID', 'LEANM', 'ALL_COHORT_1415', 'ALL_RATE_1415',
       'MAM_COHORT_1415', 'MAM_RATE_1415', 'MAS_COHORT_1415', 'MAS_RATE_1415',
       'MBL_COHORT_1415', 'MBL_RATE_1415', 'MHI_COHORT_1415', 'MHI_RATE_1415',
       'MTR_COHORT_1415', 'MTR_RATE_1415', 'MWH_COHORT_1415', 'MWH_RATE_1415',
       'CWD_COHORT_1415', 'CWD_RATE_1415', 'ECD_COHORT_1415', 'ECD_RATE_1415',
       'LEP_COHORT_1415', 'LEP_RATE_1415', 'DATE_CUR'],
      dtype='object')


In [16]:
print(A2.columns)
print(B2.columns)

Index(['STATE', 'LEAID', 'LEANM', 'Total Schools', 'Total Charter Schools',
       'Total Public Schools', 'Total Students',
       'Free and Reduced Lunch Students', 'Grade 12 Students - male',
       'Grade 12 Students - female', 'Pupil/Teacher Ratio',
       'Full-Time Equivalent', 'Secondary Teachers', 'Total Staff',
       'Secondary Guidance Counselors'],
      dtype='object')
Index(['STATE', 'LEAID', 'LEANM', 'Total Schools', 'Total Charter Schools',
       'Total Public Schools', 'Total Students',
       'Free and Reduced Lunch Students', 'Grade 12 Students - male',
       'Grade 12 Students - female', 'Pupil/Teacher Ratio',
       'Full-Time Equivalent', 'Secondary Teachers', 'Total Staff',
       'Secondary Guidance Counselors'],
      dtype='object')


In [17]:
print(A3.columns)
print(B3.columns)

Index(['STNAM', 'FIPST', 'LEAID', 'leanm11', 'ALL_MTH00numvalid_1112',
       'ALL_MTH00pctprof_1112', 'MAM_MTH00numvalid_1112',
       'MAM_MTH00pctprof_1112', 'MAS_MTH00numvalid_1112',
       'MAS_MTH00pctprof_1112',
       ...
       'MIG_MTH05numvalid_1112', 'MIG_MTH05pctprof_1112',
       'MIG_MTH06numvalid_1112', 'MIG_MTH06pctprof_1112',
       'MIG_MTH07numvalid_1112', 'MIG_MTH07pctprof_1112',
       'MIG_MTH08numvalid_1112', 'MIG_MTH08pctprof_1112',
       'MIG_MTHHSnumvalid_1112', 'MIG_MTHHSpctprof_1112'],
      dtype='object', length=228)
Index(['STNAM', 'FIPST', 'LEAID', 'LEANM', 'DATE_CUR',
       'ALL_MTH00NUMVALID_1415', 'ALL_MTH00PCTPROF_1415',
       'MAM_MTH00NUMVALID_1415', 'MAM_MTH00PCTPROF_1415',
       'MAS_MTH00NUMVALID_1415',
       ...
       'MIG_MTH05NUMVALID_1415', 'MIG_MTH05PCTPROF_1415',
       'MIG_MTH06NUMVALID_1415', 'MIG_MTH06PCTPROF_1415',
       'MIG_MTH07NUMVALID_1415', 'MIG_MTH07PCTPROF_1415',
       'MIG_MTH08NUMVALID_1415', 'MIG_MTH08PCTPROF_1415'

In [18]:
print(A4.columns)
print(B4.columns)

Index(['STNAM', 'FIPST', 'LEAID', 'leanm11', 'ALL_RLA00numvalid_1112',
       'ALL_RLA00pctprof_1112', 'MAM_RLA00numvalid_1112',
       'MAM_RLA00pctprof_1112', 'MAS_RLA00numvalid_1112',
       'MAS_RLA00pctprof_1112',
       ...
       'MIG_RLA05numvalid_1112', 'MIG_RLA05pctprof_1112',
       'MIG_RLA06numvalid_1112', 'MIG_RLA06pctprof_1112',
       'MIG_RLA07numvalid_1112', 'MIG_RLA07pctprof_1112',
       'MIG_RLA08numvalid_1112', 'MIG_RLA08pctprof_1112',
       'MIG_RLAHSnumvalid_1112', 'MIG_RLAHSpctprof_1112'],
      dtype='object', length=228)
Index(['STNAM', 'FIPST', 'LEAID', 'LEANM', 'DATE_CUR',
       'ALL_RLA00NUMVALID_1415', 'ALL_RLA00PCTPROF_1415',
       'MAM_RLA00NUMVALID_1415', 'MAM_RLA00PCTPROF_1415',
       'MAS_RLA00NUMVALID_1415',
       ...
       'MIG_RLA05NUMVALID_1415', 'MIG_RLA05PCTPROF_1415',
       'MIG_RLA06NUMVALID_1415', 'MIG_RLA06PCTPROF_1415',
       'MIG_RLA07NUMVALID_1415', 'MIG_RLA07PCTPROF_1415',
       'MIG_RLA08NUMVALID_1415', 'MIG_RLA08PCTPROF_1415'

In [19]:
#acrg2011 = parsecohortdata("data/EDFacts/regulatory-adjusted-cohort-graduation-rate/2011/acgr-lea-sy2011-12.csv", 2011)
#acrg2014 = parsecohortdata("data/EDFacts/regulatory-adjusted-cohort-graduation-rate/2014/acgr-lea-sy2014-15.csv", 2014)

#sefd2011 = parsefinancedata("data/Census/Public Elementary–Secondary Education Finance Data/elsec11t.csv")
#sefd2014 = parsefinancedata("data/Census/Public Elementary–Secondary Education Finance Data/elsec14t.csv")

#smipe2011 = parsepovertydata("data/Census/Small Area Income and Poverty Estimates/USSD11.csv")
#smipe2014 = parsepovertydata("data/Census/Small Area Income and Poverty Estimates/USSD14.csv")

elsi2011  = parseelsidata("data/NCES/2011/ELSI_csv_export_2011.csv", 2011)
elsi2014  = parseelsidata("data/NCES/2014/ELSI_csv_export_2014.csv", 2014)

admt2011 = parseassessmentdata('data/EDFacts/Assessment Proficiency/2011/math-achievement-lea-sy2011-12.csv', 2011, 'MATH')
admt2014 = parseassessmentdata('data/EDFacts/Assessment Proficiency/2014/math-achievement-lea-sy2014-15.csv', 2014, 'MATH')

adrl2011 = parseassessmentdata('data/EDFacts/Assessment Proficiency/2011/rla-achievement-lea-sy2011-12.csv', 2011, 'ENGLISH')
adrl2014 = parseassessmentdata('data/EDFacts/Assessment Proficiency/2014/rla-achievement-lea-sy2014-15.csv', 2014, 'ENGLISH')

acrg2011 = parsecohortdata("data/EDFacts/regulatory-adjusted-cohort-graduation-rate/2011/acgr-lea-sy2011-12.csv", 2011)
acrg2014 = parsecohortdata("data/EDFacts/regulatory-adjusted-cohort-graduation-rate/2014/acgr-lea-sy2014-15.csv", 2014)

temp1 = pd.merge(acrg2011, admt2011, how='inner', on='LEAID')
temp2 = pd.merge(temp1, adrl2011, how='inner', on='LEAID')
df2011 = pd.merge(temp2, elsi2011, how='inner', on='LEAID')
#df2011 = pd.merge(temp3, smipe2011, how='inner', on='LEAID')

temp4 = pd.merge(acrg2014, admt2014, how='inner', on='LEAID')
temp5 = pd.merge(temp4, adrl2014, how='inner', on='LEAID')
df2014 = pd.merge(temp5, elsi2014, how='inner', on='LEAID')
#df2014 = pd.merge(temp6, smipe2014, how='inner', on='LEAID')

print(df2011.shape)
print(df2014.shape)

df2011r = df2011[df2011['ALL_RATE'] != 0]
df2014r = df2014[df2014['ALL_RATE'] != 0]

#Write the data in a csv file
df2011r.to_csv('data/high-school-dropout-dataset2011.csv', mode = 'w', index=False)
df2014r.to_csv('data/high-school-dropout-dataset2014.csv', mode = 'w', index=False)

#Find total number of records with total number columns
print(df2011r.shape)
print(df2014r.shape)

# Check all unique values in dataframe
print(pd.unique(df2011r.ALL_RATE.ravel()))
print(pd.unique(df2014r.ALL_RATE.ravel()))

#Check if there are records with more than 100% graduation rate
print(len(df2011r[df2011r['ALL_RATE'] > 100]))
print(len(df2014r[df2014r['ALL_RATE'] > 100]))

#Check if there are records with 0% Pass rate
print(len(df2011r[df2011r['ALL_RATE'] == 0]))
print(len(df2014r[df2014r['ALL_RATE'] == 0]))

#Find records with more than 0% Pass rate
print(len(df2011r[df2011r['ALL_RATE'] < 0]))
print(len(df2014r[df2014r['ALL_RATE'] < 0]))

(11177, 50)
(11844, 50)
(11164, 50)
(11824, 50)
[83 79 91 73 93 78 76 65 87 63 88 81 74 60 40 80 56 72 66 75 67 89 92 85 71
 82 61 84 86 70 77 68 64 98 94 90 69 58 97 44 41 36 50 99 30 46 55 96 39 59
 43 27 51 52 42 54 48 14 57 20 17 21 37 33 28 22 10 24 32 26 53 15 19 95  7
 47 38 13 12 31  3 45 23 25 35 18 34 62  1 11  8  6 49  5 16  9  2 29  4]
[91 93 97 87 84 98 95 78 92 86 90 85 80 72 83 88 81 82 94 89 96 99 79 53 61
 52 44 55 71 75 77 65 54 11 58 73 30 69 36 67 62 45 63 35 76 74 43  3 17 27
 16 40 64 42 60 57 50 51 25 31  1 59 22 26 21 68 41  5  9 13 47  2 19 24 18
 23 29 33 32 34 37  4 56 48 38 10 39 70 66 28 15 46  6  7 20  8 12 14 49]
0
0
0
0
0
0


In [23]:
#Find first 5 records of 2011 regulatory adjusted cohort graduation rate
df2011r.iloc[:5,:20]

,YEAR,STATE,LEAID,ALL_COHORT,ALL_RATE,MAM_RATE,MAS_RATE,MBL_RATE,MHI_RATE,MTR_RATE,MWH_RATE,CWD_RATE,ECD_RATE,LEP_RATE,MAM_RATIO,MAS_RATIO,MBL_RATIO,MHI_RATIO,MTR_RATIO,MWH_RATIO
0,2011,ALABAMA,0100005,268,83,0,0,73,61,0,85,76,72,46,0.0,0.0,2.0,18.0,0.0,79.0
1,2011,ALABAMA,0100006,424,79,96,25,25,60,0,79,72,72,13,0.0,0.0,1.0,6.0,0.0,92.0
2,2011,ALABAMA,0100007,1042,91,68,86,85,66,48,94,72,78,31,0.0,7.0,21.0,5.0,0.0,66.0
3,2011,ALABAMA,0100008,836,91,72,91,85,76,3,93,63,82,95,0.0,5.0,23.0,3.0,0.0,68.0
4,2011,ALABAMA,0100011,117,73,0,0,62,91,0,75,71,67,48,0.0,0.0,38.0,9.0,0.0,53.0


In [21]:
#Find first 5 records of 2014 regulatory adjusted cohort graduation rate
df2014r.iloc[:5,:20]

,YEAR,STATE,LEAID,ALL_COHORT,ALL_RATE,MAM_RATE,MAS_RATE,MBL_RATE,MHI_RATE,MTR_RATE,MWH_RATE,CWD_RATE,ECD_RATE,LEP_RATE,MAM_RATIO,MAS_RATIO,MBL_RATIO,MHI_RATIO,MTR_RATIO,MWH_RATIO
0,2014,ALABAMA,0100005,252,91,0,19,98,86,53,93,1,76,64,0.0,0.0,2.0,26.0,1.0,71.0
1,2014,ALABAMA,0100006,369,91,60,0,84,90,0,89,83,86,38,1.0,0.0,1.0,11.0,0.0,87.0
2,2014,ALABAMA,0100007,1075,93,0,96,88,85,95,95,56,86,68,0.0,7.0,26.0,5.0,3.0,60.0
3,2014,ALABAMA,0100008,860,97,97,95,95,88,80,97,85,93,59,1.0,6.0,22.0,3.0,2.0,66.0
4,2014,ALABAMA,0100011,116,87,31,2,91,77,70,87,48,86,0,2.0,1.0,33.0,7.0,2.0,56.0


In [38]:
df2011r[['YEAR', 'STATE', 'LEAID', 'ALL_MTH_PCT', 'ALL_RLA_PCT']].head(5)

,YEAR,STATE,LEAID,ALL_MTH_PCT,ALL_RLA_PCT
0,2011,ALABAMA,0100005,84,84
1,2011,ALABAMA,0100006,90,82
2,2011,ALABAMA,0100007,97,94
3,2011,ALABAMA,0100008,95,93
4,2011,ALABAMA,0100011,74,84


In [39]:
df2014r[['YEAR', 'STATE', 'LEAID', 'ALL_MTH_PCT', 'ALL_RLA_PCT']].head(5)

,YEAR,STATE,LEAID,ALL_MTH_PCT,ALL_RLA_PCT
0,2014,ALABAMA,0100005,17.0,54.0
1,2014,ALABAMA,0100006,15.0,56.0
2,2014,ALABAMA,0100007,43.0,77.0
3,2014,ALABAMA,0100008,50.0,79.0
4,2014,ALABAMA,0100011,19.0,74.0
